In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import glob
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import pickle

# Parameters for feature extraction
LBP_RADIUS = 1
LBP_POINTS = 8 * LBP_RADIUS
GLCM_DISTANCES = [1, 2, 3]  # Increased distances
GLCM_ANGLES = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # Increased angles
scaler = StandardScaler()

def extract_color_histogram(image, bins=(4, 4, 4)):
    """Extract color histogram features from an image."""
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

def extract_lbp_features(image, radius=LBP_RADIUS, points=LBP_POINTS):
    """Extract Local Binary Pattern (LBP) features from an image."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(gray, points, radius, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, points + 3), range=(0, points + 2))
    hist = hist.astype('float')
    hist /= (hist.sum() + 1e-6)
    return hist

def extract_glcm_features(image, distances=GLCM_DISTANCES, angles=GLCM_ANGLES):
    """Extract Gray-Level Co-occurrence Matrix (GLCM) features from an image."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    glcm = graycomatrix(gray, distances=distances, angles=angles, symmetric=True, normed=True)
    features = []
    for prop in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']:
        feature = graycoprops(glcm, prop)
        features.extend(feature.flatten())
    return np.array(features)

def extract_invariant_moments(image):
    """Extract invariant moment features from an image."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    moments = cv2.moments(gray)
    hu_moments = cv2.HuMoments(moments).flatten()
    return hu_moments



def extract_features(image_path):
    """Extract combined features from an image."""
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Unable to read image at {image_path}")
        return None
    color_hist = extract_color_histogram(image)
    lbp_features = extract_lbp_features(image)
    glcm_features = extract_glcm_features(image)
    invariant_moments = extract_invariant_moments(image)
    
    return np.hstack([color_hist, lbp_features, glcm_features, invariant_moments]) 


In [ ]:

# Load dataset
dataset_path = 'smallerrefined'
image_paths = glob.glob(os.path.join(dataset_path, '**', '*.jpg'), recursive=True)

# Assign labels based on folder names
label_map = {'degree_0': 0, 'degree_1': 1, 'degree_2': 2}
features = []
labels = []

for path in image_paths:
    folder_name = os.path.basename(os.path.dirname(path))
    label = label_map.get(folder_name)
    if label is not None:
        feature = extract_features(path)
        if feature is not None:
            features.append(feature)
            labels.append(label)
    else:
        print(f"Warning: Folder name {folder_name} does not match any label.")

X = np.array(features)
y = np.array(labels)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM classifier
svm = SVC(kernel='linear', C=2, gamma='scale', random_state=42)
svm.fit(X_train, y_train)

# Evaluate classifier
y_pred = svm.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Feature Importance
if svm.kernel == 'linear':
    importances = np.abs(svm.coef_[0])

    # Determine feature index ranges
    color_hist_indices = range(0, 64)  # Color histogram has 64 bins
    lbp_indices = range(64, 64 + (LBP_POINTS + 2))  # LBP feature range
    glcm_indices = range(64 + (LBP_POINTS + 2), 64 + (LBP_POINTS + 2) + (5 * len(GLCM_DISTANCES) * len(GLCM_ANGLES)))  # GLCM
    hu_moment_indices = range(64 + (LBP_POINTS + 2) + (5 * len(GLCM_DISTANCES) * len(GLCM_ANGLES)), 
                              64 + (LBP_POINTS + 2) + (5 * len(GLCM_DISTANCES) * len(GLCM_ANGLES)) + 7)  # Hu Moments (7 values)

    # Aggregate importances
    color_hist_importance = np.sum(importances[list(color_hist_indices)])
    lbp_importance = np.sum(importances[list(lbp_indices)])
    glcm_importance = np.sum(importances[list(glcm_indices)])
    hu_moment_importance = np.sum(importances[list(hu_moment_indices)])

    # Combine into a dictionary for plotting
    feature_type_importances = {
        'Color Histogram': color_hist_importance,
        'LBP': lbp_importance,
        'GLCM': glcm_importance,
        'Hu Moments': hu_moment_importance
    }

    # Plot Aggregated Feature Importances
    plt.figure(figsize=(8, 6))
    plt.bar(feature_type_importances.keys(), feature_type_importances.values(), color=['blue', 'green', 'red', 'purple'])
    plt.xlabel('Feature Type')
    plt.ylabel('Aggregated Importance')
    plt.title('Aggregated Feature Importances by Type')
    plt.show()

else:
    print("Feature importances are not available for non-linear kernels.")
print(f"Hu Moment Indices: {hu_moment_indices}")
print(f"SVM Importances for Hu Moments: {importances[hu_moment_indices]}")




In [ ]:
# save the model
with open('svm_model.pkl', 'wb') as f:
    pickle.dump(svm, f)

In [ ]:
import pickle

# Path to the new image
# Load the saved SVM model
with open('svm_model.pkl', 'rb') as model_file:
    loaded_svm = pickle.load(model_file)

new_image_path = r'C:/Users/husse/Desktop/All Courses/GP/El7a2ny/SkinBurns/refineddataset/degree_2/img225_crop_0.jpg'

# Extract features from the new image
new_image_features = extract_features(new_image_path)

if new_image_features is not None:
    # Reshape the features to match the input format of the SVM model
    new_image_features = new_image_features.reshape(1, -1)
    
    # Predict the class using the loaded model
    predicted_class = loaded_svm.predict(new_image_features)
    print(f"Predicted class: {predicted_class[0]}")
else:
    print("Failed to extract features from the image.")